In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [1]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSORIO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: Limbo - UTPS>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b

In [2]:
import json
from helpers.fields import FeatureCollectionField
import shapefile as shpfile
from tqdm.notebook import tqdm
from helpers.histories import bulk_update_with_history
from collections import defaultdict
import random

In [3]:
def get_uf(uf_name):
    if type(uf_name) != 'str':
        uf_code=str(uf_name)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'},
       {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'},
       {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'},
       {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'},
       {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
       {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
       {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    
    uf_translation={item['name']: item['id'] for item in uf}

    return uf_translation[uf_name]

# Escolher unidade

In [4]:
company = Company.objects.get(name__startswith='GES - UHE-ESTREITO')
engie=User.objects.get(username='engie')
company, engie

(<Company: 42ddc389-3258-4daf-868a-097235427e0f: GES - UHE-ESTREITO>,
 <User: engie - Suporte Kartado>)

# Escolher nome do arquivo

In [8]:
folder = "Mosaico/"
file_name = "Mosaico_fundiario_UHE_V08022023"
reader = shpfile.Reader(folder + file_name + ".shp", encoding = "latin-1")

In [9]:
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []

In [10]:
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append({"type":"Feature", "geometry":geom, "properties":atr}) 

    
geojson = {"type": "FeatureCollection","features": buffer}

Exception: Shape type "NULL" cannot be represented as GeoJSON.

# Mudanças em registros (começo) -> opcional

### Parte 1

In [149]:
# reps=[]

# for item in tqdm(buffer):
#     reg=OccurrenceRecord.objects.get(company=uhit, number=item['properties']['NUMREG'])
#     reg.uf = get_uf(item['properties']['SIGLA'])
# #     reg.place_on_dam=item['properties']['ZONA']
    
#     try:
#         reg.city = City.objects.get(name=item['properties']['NM_MUN'], uf_code=reg.uf)
#     except Exception as e:
#         new_city = City(name=item['properties']['NM_MUN'], uf_code=reg.uf)
#         new_city.save()
#         reg.city=new_city
        
#     reps.append(reg)

In [150]:
# regs_update = bulk_update_with_history(reps, OccurrenceRecord, user=engie, use_django_bulk=True, batch_size=100)

### Parte 2

In [151]:
# regs=defaultdict(list)

# for item in tqdm(buffer):
#     regs[item['properties']['NUMREG']].append(item['properties']['WGS84_Z'])

In [152]:
# reps=[]

# for k, v in tqdm(regs.items()):
#     try:
#         reg=OccurrenceRecord.objects.get(company=company, number=k)
#         reg.form_data['altitude']=min(v)
#         reps.append(reg)

#     except Exception as e:
# #         try:
# #             reg=OccurrenceRecord.objects.get(company=company, number=k.replace("UHIT", "UHSS"))
# #             reg.form_data['altitude']=min(v)
# #             reps.append(reg)
# #         except Exception as ee:
#         print(k)

In [153]:
# regs_update = bulk_update_with_history(reps, OccurrenceRecord, user=engie, use_django_bulk=True, batch_size=100)

### Parte 3

In [154]:
# regs=defaultdict(list)

# for item in tqdm(buffer):
#     regs[item['properties']['NUMREG'].replace("UHIT", "UHSS")].append([item['properties']['NM_MUN'].encode('latin-1').decode('UTF-8'), item['properties']['SIGLA']])

In [155]:
# for k, v in tqdm(regs.items()):
#     if len(v) > 2:
#         print(k, v)

In [156]:
# reps=[]

# for k, v in tqdm(regs.items()):
#     reg=OccurrenceRecord.objects.get(company=company, number=k)
#     try:
#         reg.uf = get_uf(v[0][1].strip())
#         reg.city=City.objects.get(name=v[0][0].strip())
#         reps.append(reg)
#     except Exception as e:
#         try:
# #             reg=OccurrenceRecord.objects.get(company=company, number=k.replace("UHIT", "UHSS"))
# #             reg.uf = get_uf(v[0][1].strip())
#             new_city = City(name=v[0][0].strip(), uf_code=reg.uf)
#             new_city.save()
#             reg.city=new_city
#             reps.append(reg)
#         except Exception as ee:
#             print(ee)

In [157]:
# regs_update = bulk_update_with_history(reps, OccurrenceRecord, user=engie, use_django_bulk=True, batch_size=100)

# Mudanças em registros (fim)

# Avisos abaixo

In [158]:
# OBS: Necessário manipular internal_values['properties']
# Cadastramento = Propriedades para cadastro #FF0000


# OBS: Não é necessário manipular internal_values['properties']
# ZSS = Limite da ZSS #D8BDD8 -> enable_default=True
# ZAS = Limite da ZAS #AF2020 -> enable_default=True
# Malha = Malha Viária #FFEB3B
# Pontos de encontro = Pontos de encontro #01DF01
# Rotas de fuga = Rotas de Fuga #FF0000

In [159]:
color = "#dbf0ff"

## Nome da camada no sistema

In [160]:
nome = file_name

## Importando a camada no sistema

In [161]:
field = FeatureCollectionField(
    required=False,
    allow_null=True,
    geometry_field="geometry",
    properties_field="properties",
)


internal_values = field.to_internal_value(geojson)

"""Linha opcional abaixo, de acordo com tipo de shape"""
# internal_values['properties'] = [{"NUMCAD": a['NUMCAD'], "id": a['NUMCAD'], "name": a['NUMCAD'], "label": a['NUMCAD'], "OBJECTID": a['NUMCAD']} for a in internal_values['properties']]

shapefile = ShapeFile(
  created_by = engie,
  name = nome,
  description = nome,
  enable_default=False,
  geometry = internal_values['geometry'],
  properties = internal_values['properties'],
  metadata = {
    "fill_color": color, # escolher cores
    "stroke_color": color # escolher cores
}
)

shapefile.save()

# shapefile.companies.add(company)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ShapeFile.synced_at received a naive datetime (2023-10-23 14:15:00.793387) while time zone support is active.
  RuntimeWarning)


In [162]:
shapefile

<ShapeFile: [] SC_Municipios_2022>